## Advanced Box Score Calculations
Author: Akshay Easwaran <akeaswaran@me.com>

---
Based on work by ESPN's Bill Connelly, Football Outsiders, and Football Study Hall

Data from CollegeFootballData.com

Useful Resources for Stat Definitions:
- Football Study Hall website: https://www.footballstudyhall.com/
- Football Study Hall book: https://www.amazon.com/Study-Hall-College-Football-Stories/dp/1484989961
- Football Outsiders: https://www.footballoutsiders.com/info/glossary
- Example Advanced Box Score: https://twitter.com/ESPN_BillC/status/1176572006969597954/photo/1

**How to use this Notebook**

1. Dump a game's play by play data and drive data from CollegeFootballData.com.
2. Change the file names in the next cell to match where your data files are located.
3. Change the away and home team names appropriately.
4. Hit Cell > Run All Cells.

In [1]:
import pandas as pd

pbp_data = pd.read_json('data/pbp/pbp-data-gatech-duke.json')
# pbp_data.info()

drives = pd.read_json('data/drives/drive-data-gatech-duke.json')
drives.drop(['offense_conference','start_time','end_time','defense_conference','elapsed','game_id'], axis = 1, inplace=True) 
drives = drives[
    (~drives.drive_result.isin(['END OF HALF','END OF GAME','Uncategorized']))
    & (~drives.drive_result.str.contains('END OF'))
]

away_team = 'Georgia Tech'
home_team = 'Duke'

In [2]:
# Data Cleaning
# Fix the bad yard line markers for away teams
drives.loc[
    drives.offense == away_team, ['start_yardline']
] = 100 - drives.start_yardline
drives.loc[
    drives.offense == away_team, ['end_yardline']
] = 100 - drives.end_yardline

print("Total Drives:", len(drives))

# Eliminate garbage time plays
garbage_time_data = pbp_data[
    (pbp_data.down == 0)
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 38) & (pbp_data.period == 2))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 28) & (pbp_data.period == 3))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 22) & (pbp_data.period == 4))
]

print("Total Plays:", len(pbp_data))
print("Garbage Time Plays:", len(garbage_time_data))
pbp_data.drop(garbage_time_data.index, inplace=True)
print("Non-Garbage Time Plays:", len(pbp_data))

# Drop some unnecessary columns
pbp_data.drop(['offense_conference','defense_conference','clock'], axis = 1, inplace=True) 
# Ignore some types of plays cause they're special teams and weird
ignore_types = ["Defensive 2pt Conversion","Blocked Field Goal","Blocked Punt","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]
pbp_data = pbp_data[~(pbp_data.play_type.isin(ignore_types))]
print("Total clean plays:", len(pbp_data))

Total Drives: 23
Total Plays: 194
Garbage Time Plays: 16
Non-Garbage Time Plays: 178
Total clean plays: 140


In [3]:
import numpy as np

pbp_data.distance = pbp_data.distance.astype(float)

bad_types = ["Interception","Pass Interception Return","Interception Return Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

def is_successful(down, distance, yards_gained, play_type):
    if (play_type in bad_types):
        return False 
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif ((down == 3) & (yards_gained >= distance)):
        return True
    elif ((down == 4) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp_data['play_successful'] = np.vectorize(is_successful)(pbp_data.down, pbp_data.distance, pbp_data.yards_gained, pbp_data.play_type)
pbp_data.play_successful.value_counts()

def is_explosive(yards_gained):
    if (yards_gained >= 15):
        return True
    else:
        return False
    
pbp_data['play_explosive'] = np.vectorize(is_explosive)(pbp_data.yards_gained)

In [4]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown","Interception","Pass Interception Return","Interception Return Touchdown"]
rush_types = ["Rush","Rushing Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

print("Success Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

Success Rates
Georgia Tech
Overall: 0.3382352941176471
Passing: 0.2727272727272727
Rushing: 0.42424242424242425
Duke
Overall: 0.4305555555555556
Passing: 0.42105263157894735
Rushing: 0.4339622641509434


In [5]:
# Explosiveness rates
# Defined as rate of 15+ yards gains

print("Explosiveness Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

Explosiveness Rates
Georgia Tech
Overall: 0.16176470588235295
Passing: 0.21212121212121213
Rushing: 0.12121212121212122
Duke
Overall: 0.09722222222222222
Passing: 0.3157894736842105
Rushing: 0.018867924528301886


In [6]:
# Havoc plays
havoc_plays = pbp_data[
    (((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type == 'Fumble Recovery (Opponent)')
    | (pbp_data.play_type == 'Sack')
    | (pbp_data.play_type.str.contains('Interception', regex=False))
    | (pbp_data.yards_gained < 0))
    & (pbp_data.play_type != 'Penalty')
]

print(away_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == away_team
]), "/", len(pbp_data[
    pbp_data.defense == away_team
]), "(",len(havoc_plays[
    havoc_plays.defense == away_team
]) / len(pbp_data[
    pbp_data.defense == away_team
]),")")

print(home_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == home_team
]), "/", len(pbp_data[
    pbp_data.defense == home_team
]), "(",len(havoc_plays[
    havoc_plays.defense == home_team
]) / len(pbp_data[
    pbp_data.defense == home_team
]),")")

Georgia Tech Havoc Rate:  7 / 72 ( 0.09722222222222222 )
Duke Havoc Rate:  12 / 68 ( 0.17647058823529413 )


In [7]:
# Calculating converted 3rd and 8+s
ignore_turnovers = ['Fumble Recovery (Opponent)','Fumble Return Touchdown',"Interception","Pass Interception Return","Interception Return Touchdown"]

third_longs = pbp_data[
    (pbp_data.down == 3)
    & (pbp_data.distance >= 8)
    & (~pbp_data.play_type.isin(ignore_turnovers))
]
# converted_3rd_longs

away_team_thirds = third_longs[
    (third_longs.offense == away_team)
]

away_team_thirds_converted = away_team_thirds[
    (away_team_thirds.yards_gained >= 8)
]
print("Converted 3rd and 8+ for", away_team,":",len(away_team_thirds_converted),"of",len(away_team_thirds))

home_team_thirds = third_longs[
    (third_longs.offense == home_team)
]

home_team_thirds_converted = home_team_thirds[
    (home_team_thirds.yards_gained >= 8)
]

print("Converted 3rd and 8+ for", home_team,":",len(home_team_thirds_converted),"of",len(home_team_thirds))

Converted 3rd and 8+ for Georgia Tech : 3 of 10
Converted 3rd and 8+ for Duke : 0 of 1


In [8]:
away_team_drives = drives[
    drives.offense == away_team
]

print(away_team, "Drives:", len(away_team_drives))
print(away_team,"Yards:",sum(away_team_drives.yards))
print(away_team,"Plays:",sum(away_team_drives.plays))
print(away_team, "Avg Starting Field Position:", sum(away_team_drives.start_yardline) / len(away_team_drives))
print(away_team, "Yards per Play:", sum(away_team_drives.yards) / sum(away_team_drives.plays))
print(away_team, "Plays per Drive:", sum(away_team_drives.plays) / len(away_team_drives))
print(away_team, "Yards per Drive:", sum(away_team_drives.yards) / len(away_team_drives))
away_team_drives

Georgia Tech Drives: 12
Georgia Tech Yards: 291
Georgia Tech Plays: 66
Georgia Tech Avg Starting Field Position: 25.416666666666668
Georgia Tech Yards per Play: 4.409090909090909
Georgia Tech Plays per Drive: 5.5
Georgia Tech Yards per Drive: 24.25


,defense,drive_result,end_period,end_yardline,id,offense,plays,scoring,start_period,start_yardline,yards
0,Duke,TD,1,100,4011124801,Georgia Tech,11,True,1,25,75
2,Duke,PUNT,1,8,4011124803,Georgia Tech,3,False,1,18,-10
4,Duke,PUNT,1,37,4011124805,Georgia Tech,5,False,1,25,12
6,Duke,PUNT,2,27,4011124807,Georgia Tech,3,False,2,27,0
8,Duke,DOWNS,2,31,4011124809,Georgia Tech,4,False,2,25,6
10,Duke,PUNT TD,2,25,40111248011,Georgia Tech,3,True,2,25,0
11,Duke,TD,2,100,40111248013,Georgia Tech,6,True,2,25,75
14,Duke,FG,3,75,40111248016,Georgia Tech,7,True,3,11,64
16,Duke,DOWNS,3,61,40111248018,Georgia Tech,10,False,3,20,41
18,Duke,PUNT,3,18,40111248020,Georgia Tech,3,False,3,25,-7


In [9]:
home_team_drives = drives[
    drives.offense == home_team
]

print(home_team, "Drives:", len(home_team_drives))
print(home_team,"Yards:",sum(home_team_drives.yards))
print(home_team,"Plays:",sum(home_team_drives.plays))
print(home_team, "Avg Starting Field Position:", sum(home_team_drives.start_yardline) / len(home_team_drives))
print(home_team, "Yards per Play:", sum(home_team_drives.yards) / sum(home_team_drives.plays))
print(home_team, "Plays per Drive:", sum(home_team_drives.plays) / len(home_team_drives))
print(home_team, "Yards per Drive:", sum(home_team_drives.yards) / len(home_team_drives))
home_team_drives

Duke Drives: 11
Duke Yards: 382
Duke Plays: 75
Duke Avg Starting Field Position: 37.54545454545455
Duke Yards per Play: 5.093333333333334
Duke Plays per Drive: 6.818181818181818
Duke Yards per Drive: 34.72727272727273


,defense,drive_result,end_period,end_yardline,id,offense,plays,scoring,start_period,start_yardline,yards
1,Georgia Tech,FG,1,85,4011124802,Duke,11,True,1,25,60
3,Georgia Tech,TD,1,100,4011124804,Duke,13,True,1,45,55
5,Georgia Tech,TD,2,100,4011124806,Duke,8,True,2,41,59
7,Georgia Tech,TD,2,100,4011124808,Duke,8,True,2,18,82
9,Georgia Tech,TD,2,100,40111248010,Duke,4,True,2,69,31
13,Georgia Tech,PUNT,3,37,40111248015,Duke,3,False,3,30,7
15,Georgia Tech,PUNT,3,49,40111248017,Duke,6,False,3,35,14
17,Georgia Tech,FG,3,82,40111248019,Duke,8,True,3,39,43
19,Georgia Tech,PUNT,4,50,40111248021,Duke,3,False,4,51,-1
21,Georgia Tech,PUNT,4,53,40111248023,Duke,8,False,4,25,28


In [10]:
# Measuring success rate for a single player
# pbp_data[
#     (pbp_data.play_text.str.contains("Quentin Harris"))
#     & (pbp_data.play_type.isin(pass_types))
#     & (~pbp_data.play_type.str.contains("Sack"))
# ].play_successful.value_counts(normalize=True)

In [11]:
# Standard vs Passing Downs success rates
# Success rate on standard downs == leverage rate

standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]

pass_plays = pbp_data[
    pbp_data.play_type.isin(pass_types)
]
rush_plays = pbp_data[
    pbp_data.play_type.isin(rush_types)
]

In [32]:
print(away_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == away_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == away_team)
]))

print(away_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == away_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == away_team)
]))

Georgia Tech Success Rate on Std Downs: 0.4146341463414634
Georgia Tech Success Rate on Passing Downs: 0.2222222222222222


In [33]:
print(home_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == home_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == home_team)
]))

print(home_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == home_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == home_team)
]))

Duke Success Rate on Std Downs: 0.4489795918367347
Duke Success Rate on Passing Downs: 0.391304347826087


In [30]:
# Stuff Rate
stuffed_plays = rush_plays[
    (rush_plays.yards_gained <= 0)
    & (rush_plays.play_type != 'Sack')
]

away_team_stuffs = stuffed_plays[
    stuffed_plays.defense == away_team
]
print(away_team,"Defensive Stuff Rate: ",len(away_team_stuffs),"/",len(rush_plays[rush_plays.defense == away_team]),"(",len(away_team_stuffs)/len(rush_plays[rush_plays.defense == away_team]),")")

home_team_stuffs = stuffed_plays[
    stuffed_plays.defense == home_team
]
print(home_team,"Defensive Stuff Rate: ",len(home_team_stuffs),"/",len(rush_plays[rush_plays.defense == home_team]),"(",len(home_team_stuffs)/len(rush_plays[rush_plays.defense == home_team]),")")

Georgia Tech Defensive Stuff Rate:  5 / 53 ( 0.09433962264150944 )
Duke Defensive Stuff Rate:  6 / 33 ( 0.18181818181818182 )


In [31]:
# Opportunity Rate
rush_opps = rush_plays[
    (rush_plays.yards_gained >= 5)
]
print(away_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == away_team]),"/",len(rush_plays[rush_plays.offense == away_team]),"(",len(rush_opps[rush_opps.offense == away_team])/len(rush_plays[rush_plays.offense == away_team]),")")

print(home_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == home_team]),"/",len(rush_plays[rush_plays.offense == home_team]),"(",len(rush_opps[rush_opps.offense == home_team])/len(rush_plays[rush_plays.offense == home_team]),")")

Georgia Tech Rush Opp Rate:  13 / 33 ( 0.3939393939393939 )
Duke Rush Opp Rate:  17 / 53 ( 0.32075471698113206 )


In [21]:
# Line Yards -- not adjusted for down/distance/opponent/shotgun
def adjust_strength_for_ol(yards_gained):
    if (yards_gained < 0):
        return yards_gained * 1.25
    elif (yards_gained >= 0 & yards_gained <= 3):
        return yards_gained * 1.0
    elif (yards_gained >= 4 & yards_gained <= 6):
        return yards_gained * 0.5
    else:
        return 0

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - x.line_yards, axis=1)
print(away_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].line_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].highlight_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].yards_gained) / len(rush_plays[rush_plays.offense == away_team]))
print(home_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].line_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].highlight_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].yards_gained) / len(rush_plays[rush_plays.offense == home_team]))

Georgia Tech Line Yards per Carry: 4.7727272727272725
Georgia Tech Highlight Yards per Carry: 0.13636363636363635
Georgia Tech Yards per Carry: 4.909090909090909
Duke Line Yards per Carry: 3.707547169811321
Duke Highlight Yards per Carry: 0.02830188679245283
Duke Yards per Carry: 3.7358490566037736


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [22]:
# Expected and Adjusted Turnover Margins

adj_turnover_plays = pbp_data[
    (pbp_data.play_type.str.contains('Interception', regex=False))
    | ((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type.str.contains('Fumble', regex=False))
]

adj_fum = 0.5 * len(adj_turnover_plays[
    (adj_turnover_plays.play_type.str.contains('Fumble', regex=False))
])

# away_team Adj Turnovers
away_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == away_team)
    | (adj_turnover_plays.defense == away_team)
]

away_team_ints_def = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_ints_off = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_pds = len(away_team_tos[
   (away_team_tos.play_type == 'Pass Incompletion')
    & (away_team_tos.play_text.str.contains('broken up', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_fum_rec = away_team_tos[((away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.defense == away_team))]
away_team_fum_lost = away_team_tos[(away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.offense == away_team)]

In [23]:
# home_team Adj Turnovers (broken)
home_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == home_team)
    | (adj_turnover_plays.defense == home_team)
]

home_team_ints_def = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_ints_off = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_pds = len(home_team_tos[
   (home_team_tos.play_type == 'Pass Incompletion')
    & (home_team_tos.play_text.str.contains('broken up', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_fum_rec = home_team_tos[((home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.defense == home_team))]
home_team_fum_lost = home_team_tos[(home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.offense == home_team)]

print(away_team)
print("Def INTs:",away_team_ints_def)
print("Off INTs:",away_team_ints_off)
print("Def PDs:",(away_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (away_team_pds + away_team_ints_def))
print("Fum Recovered:",len(away_team_fum_rec))
print("Fum Lost:",len(away_team_fum_lost))
print("Exp Fum:",adj_fum)
print("Actual TO:",away_team_ints_off + len(away_team_fum_lost))
print("Actual TO Margin:",-1 * away_team_ints_off - len(away_team_fum_lost) + away_team_ints_def + len(away_team_fum_rec))
away_team_exp_to = 0.22 * (away_team_pds + away_team_ints_def) + adj_fum
print("Exp TO:",away_team_exp_to)

print("")

print(home_team)
print("Def INTs:",home_team_ints_def)
print("Off INTs:",home_team_ints_off)
print("Def PDs:",(home_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (home_team_pds + home_team_ints_def))
print("Fum Recovered:",len(home_team_fum_rec))
print("Fum Lost:",len(home_team_fum_lost))
print("Exp Fum:",adj_fum)
print("Actual TO:",home_team_ints_off + len(home_team_fum_lost))
print("Actual TO Margin:",-1 * home_team_ints_off - len(home_team_fum_lost) + home_team_ints_def + len(home_team_fum_rec))
home_team_exp_to = 0.22 * (home_team_pds + home_team_ints_def) + adj_fum
print("Exp TO:",home_team_exp_to)

Georgia Tech
Def INTs: 0
Off INTs: 0
Def PDs: 3
Exp INTs: 0.66
Fum Recovered: 0
Fum Lost: 0
Exp Fum: 1.0
Actual TO: 0
Actual TO Margin: 0
Exp TO: 1.6600000000000001

Duke
Def INTs: 0
Off INTs: 0
Def PDs: 2
Exp INTs: 0.44
Fum Recovered: 0
Fum Lost: 0
Exp Fum: 1.0
Actual TO: 0
Actual TO Margin: 0
Exp TO: 1.44


In [24]:
# home_team_tos

In [25]:
away_team_exp_to_margin = (away_team_exp_to - home_team_exp_to)
print("Exp TO Margin for",away_team,":", away_team_exp_to_margin)
home_team_exp_to_margin = (home_team_exp_to - away_team_exp_to)
print("Exp TO Margin for",home_team,":", home_team_exp_to_margin)

exp_to_margin = (0.22 * ((away_team_pds + away_team_ints_def) + (home_team_pds + home_team_ints_def))) + adj_fum
exp_to_margin

Exp TO Margin for Georgia Tech : 0.2200000000000002
Exp TO Margin for Duke : -0.2200000000000002


2.1

In [26]:
print("TO Luck (pts) for",away_team,":", (away_team_ints_def + len(away_team_fum_rec) - away_team_ints_off - len(away_team_fum_lost) - exp_to_margin) * 5.0)
print("TO Luck (pts) for",home_team,":", (home_team_ints_def + len(home_team_fum_rec) - home_team_ints_off - len(home_team_fum_lost) - exp_to_margin) * 5.0)


TO Luck (pts) for Georgia Tech : -10.5
TO Luck (pts) for Duke : -10.5


In [27]:
# Scoring Opportunities
# Definition: roughly, any time you get inside the opponent's 35, you should probably score

scoring_opps = drives[
    ((drives.start_yardline + drives.yards) >= 60)
]

# away team's scoring opps
print("Scoring Opportunities (IE: Drives inside Opponent's 40)")
print(away_team)
away_team_scoring_opps = scoring_opps[
    scoring_opps.offense == away_team
]
print("Total:",len(away_team_scoring_opps))
print("Scored:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]) / len(away_team_scoring_opps))
print("Opps/Drive:",len(away_team_scoring_opps) / len(drives[drives.offense == away_team]))
print("Points/Opp:",(len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'TD']) * 7 + len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'FG'] * 3)) / len(away_team_scoring_opps))

print("")
# home team's scoring opps
print(home_team)
home_team_scoring_opps = scoring_opps[
    scoring_opps.offense == home_team
]
print("Total:",len(home_team_scoring_opps))
print("Scored:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]) / len(home_team_scoring_opps))
print("Opps/Drive:",len(home_team_scoring_opps) / len(drives[drives.offense == home_team]))
print("Points/Opp:",(len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'TD']) * 7 + len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'FG'] * 3)) / len(home_team_scoring_opps))

Scoring Opportunities (IE: Drives inside Opponent's 40)
Georgia Tech
Total: 5
Scored: 4
Opp Efficiency: 0.8
Opps/Drive: 0.4166666666666667
Points/Opp: 4.4

Duke
Total: 6
Scored: 6
Opp Efficiency: 1.0
Opps/Drive: 0.5454545454545454
Points/Opp: 5.0


In [28]:
# Rushing Stats
rush_carries = rush_plays[
    (~rush_plays.play_type.isin(['Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']))
]

# look these up manually
away_team_rushers = ["Kenny Pickett"]
home_team_rushers = ["Quentin Harris"]

def verify_division(num1, num2):
    if num2 == 0:
        return 0
    else:
        return num1 / num2

print(away_team,"Rushing Stats")
for qb in away_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True)]), len(pbp_data[(pbp_data.play_text.str.contains(qb))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")

print("")

print(home_team,"Rushing Stats")
for qb in home_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True)]), len(pbp_data[(pbp_data.play_text.str.contains(qb))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")


Georgia Tech Rushing Stats
Kenny Pickett: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )

Duke Rushing Stats
Quentin Harris: 11 car 40 yards 1 TDs 0 Fum ( 3.6363636363636362 YPC , 3.6363636363636362 line YPC , SR%: 0.5161290322580645 , Opp Rate: 0.45454545454545453 , Hlt/Opp: 8.0 )


In [29]:
# Passing Stats
pass_attempts = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
    | (pass_plays.play_type == 'Pass Incompletion')
    | (pass_plays.play_type.str.contains('Interception'))
]

pass_completions = pass_attempts[
    (pass_attempts.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
]

# look these up manually
away_team_qbs = ["Ryan Hilinski","Dakereon Joyner"]
home_team_qbs = ["Jake Fromm"]

print(away_team,"Passing Stats")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")
print("")
print(home_team,"Passing Stats")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")


Georgia Tech Passing Stats
Ryan Hilinski: 0 / 0 0 yards 0 TDs 0 INTs
Dakereon Joyner: 0 / 0 0 yards 0 TDs 0 INTs

Duke Passing Stats
Jake Fromm: 0 / 0 0 yards 0 TDs 0 INTs
